In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import requests
import os
import time
import json
import math
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import statsmodels.tools.tools as stattools
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
#List of S&P 500 tickers from: https://stockmarketmba.com/stocksinthesp500.php

ticker_list = pd.read_csv('SP500_index.csv')['Symbol']
ticker_list

0       AAPL
1       MSFT
2       GOOG
3      GOOGL
4       AMZN
       ...  
498     LUMN
499      ALK
500      NWL
501      VNO
502      TAP
Name: Symbol, Length: 503, dtype: object

In [3]:
def income_puller(ticker):
    url=f'https://stockrow.com/api/companies/{ticker}/financials.xlsx?dimension=T&section=Income%20Statement&sort=desc'
    response = requests.get(url)
    with open(os.path.join("StockRow_financials/Excel", f"{ticker}_inc.xlsx"), 'wb') as f:
        f.write(response.content)
    time.sleep(0.25)

def csv_maker(ticker):
    #this section reads the Excel file and turns into a CSV
    ticker_df = pd.read_excel(f'StockRow_financials/Excel/{ticker}_inc.xlsx').T
    # change the first row to the columns
    ticker_df.columns = ticker_df.iloc[0]
    #remove the first row
    ticker_df = ticker_df.iloc[1:]
    
    ticker_df['Ticker'] = f"{ticker}"
    
    ticker_df.to_csv(f'StockRow_financials/CSV/{ticker}_inc.csv',index_label="Date")

#### This code pulls financial information from StockRow.com

In [4]:
#do not call this function unless you have 15 minutes to kill
#also note that it overwrites all the previously downloaded Excel workbooks
#start_time = time.time()
#
#for ticker in ticker_list:
#    try:
#        income_puller(ticker)
#    except:
#        print(f"Ticker {ticker} throws an error")
#    
#print(f"Code took {np.round(time.time() - start_time,2)} seconds to run")

#### This code converts the StockRow workbooks from Excel files to CSV files

In [5]:
start_time = time.time()

for ticker in ticker_list:
    try:
        csv_maker(ticker)
    except:
        print(f"Ticker {ticker} throws an error")
    
print(f"Code took {np.round(time.time() - start_time,2)} seconds to run")

Ticker META throws an error
Ticker ELV throws an error
Ticker BALL throws an error
Ticker TRGP throws an error
Ticker GEN throws an error
Code took 10.89 seconds to run


#### This code combines all downloaded CSV files (for which we have a ticker) into one Pandas DataFrame

In [6]:
starter_df = pd.read_csv(f'StockRow_financials/CSV/{ticker_list[0]}_inc.csv')

start_time = time.time()

for ticker in ticker_list[1:]:
    try:        
        ticker_df = pd.read_csv(f'StockRow_financials/CSV/{ticker}_inc.csv')
        starter_df = pd.concat([starter_df,ticker_df])
    except:
        print(f'Ticker {ticker} threw an error')
    
        
        
print(f"This code took {time.time() - start_time} seconds")

Ticker META threw an error
Ticker ELV threw an error
Ticker BALL threw an error
Ticker TRGP threw an error
Ticker GEN threw an error
This code took 1.9608309268951416 seconds


In [7]:
#save the DataFrame as one large CSV
starter_df.to_csv('SP500_allFinancials.csv',index=False)

#### Narrow the financial columns, convert "Date" to a DateTime object, create a "Year" field

In [8]:
starter_df = starter_df[['Ticker', 'Date', 'Revenue', 'Gross Profit',
                         'Operating Income', 'Income after Tax', 'Net Income Common']].reset_index(drop=True)

starter_df['Date'] = pd.to_datetime(starter_df['Date'])
starter_df['Year'] = pd.DatetimeIndex(starter_df['Date']).year

In [9]:
#example of grouping the data the find the average financial values for each calendar year
starter_df[starter_df['Ticker']=='AAPL'].groupby('Year').mean()

Revenue  Gross Profit  Operating Income  Income after Tax  \
Year                                                                   
2012  1.646870e+11  6.901900e+10      5.511100e+10      4.174700e+10   
2013  1.708525e+11  6.525800e+10      5.011225e+10      3.787200e+10   
2014  1.841935e+11  7.101400e+10      5.336100e+10      4.005850e+10   
2015  2.263010e+11  9.029050e+10      6.853200e+10      5.141750e+10   
2016  2.203950e+11  8.646250e+10      6.224650e+10      4.734475e+10   
2017  2.280935e+11  8.771550e+10      6.122750e+10      4.781425e+10   
2018  2.574745e+11  9.855900e+10      6.820600e+10      5.710000e+10   
2019  2.613452e+11  9.912250e+10      6.499925e+10      5.641200e+10   
2020  2.776220e+11  1.064325e+11      6.831750e+10      5.924500e+10   
2021  3.541752e+11  1.469285e+11      1.036732e+11      8.958700e+10   
2022  3.892957e+11  1.686247e+11      1.190483e+11      1.004570e+11   

      Net Income Common  
Year                     
2012       4.174700e+10  
2013       3.787200e+10  
2014       4.005850e+10  
2015       5.141750e+10  
2016       4.734475e+10  
2017       4.781425e+10  
2018       5.710000e+10  
2019       5.641200e+10  
2020       5.924500e+10  
2021       8.958700e+10  
2022       1.004570e+11

In [10]:
#grouping the data to find average values per year for every ticker
condensed_finances_df = starter_df.groupby(['Ticker','Year']).mean()
condensed_finances_df

Revenue  Gross Profit  Operating Income  Income after Tax  \
Ticker Year                                                                   
A      2013  6.854500e+09  3.569250e+09      9.845000e+08      9.442500e+08   
       2014  5.252250e+09  2.680750e+09      6.885000e+08      4.702500e+08   
       2015  4.619250e+09  2.308250e+09      5.257500e+08      3.070000e+08   
       2016  4.116000e+09  2.130250e+09      5.877500e+08      4.672500e+08   
       2017  4.357750e+09  2.319750e+09      7.500000e+08      6.020000e+08   
...                   ...           ...               ...               ...   
ZTS    2018  5.644000e+09  3.819000e+09      1.673500e+09      1.166000e+09   
       2019  6.092500e+09  4.097500e+09      1.727250e+09      1.430500e+09   
       2020  6.474000e+09  4.500250e+09      1.967500e+09      1.631250e+09   
       2021  7.454000e+09  5.169500e+09      2.354000e+09      1.922500e+09   
       2022  7.964333e+09  5.451667e+09      2.553333e+09      2.074000e+09   

             Net Income Common  
Ticker Year                     
A      2013       9.442500e+08  
       2014       6.605000e+08  
       2015       3.390000e+08  
       2016       4.650000e+08  
       2017       6.020000e+08  
...                        ...  
ZTS    2018       1.171250e+09  
       2019       1.431000e+09  
       2020       1.632250e+09  
       2021       1.925500e+09  
       2022       2.076667e+09  

[5330 rows x 5 columns]

In [11]:
#sanity check using AAPL
condensed_finances_df.loc['AAPL']

Revenue  Gross Profit  Operating Income  Income after Tax  \
Year                                                                   
2012  1.646870e+11  6.901900e+10      5.511100e+10      4.174700e+10   
2013  1.708525e+11  6.525800e+10      5.011225e+10      3.787200e+10   
2014  1.841935e+11  7.101400e+10      5.336100e+10      4.005850e+10   
2015  2.263010e+11  9.029050e+10      6.853200e+10      5.141750e+10   
2016  2.203950e+11  8.646250e+10      6.224650e+10      4.734475e+10   
2017  2.280935e+11  8.771550e+10      6.122750e+10      4.781425e+10   
2018  2.574745e+11  9.855900e+10      6.820600e+10      5.710000e+10   
2019  2.613452e+11  9.912250e+10      6.499925e+10      5.641200e+10   
2020  2.776220e+11  1.064325e+11      6.831750e+10      5.924500e+10   
2021  3.541752e+11  1.469285e+11      1.036732e+11      8.958700e+10   
2022  3.892957e+11  1.686247e+11      1.190483e+11      1.004570e+11   

      Net Income Common  
Year                     
2012       4.174700e+10  
2013       3.787200e+10  
2014       4.005850e+10  
2015       5.141750e+10  
2016       4.734475e+10  
2017       4.781425e+10  
2018       5.710000e+10  
2019       5.641200e+10  
2020       5.924500e+10  
2021       8.958700e+10  
2022       1.004570e+11

In [12]:
execpay_df = pd.read_csv('main_SCT_pay_file.csv')

execpay_df = execpay_df[execpay_df['ticker'].isin(ticker_list)]

execpay_df

id      cik ticker                 name  \
0       73b3a60ba203743c008330c96b7d8b66  1090872      A             Sam Raha   
1       97393f60cd9f321650e472673daaa70c  1090872      A  Michael R. McMullen   
2       83b9cc2bca477fe8ce23e0ab56e70c66  1090872      A        Jacob Thaysen   
3       7c60bb804071675ae15ec930f6dea190  1090872      A       Robert McMahon   
4       259b710a8befe67c61a2c3dec14f344a  1090872      A    Padraig McDonnell   
...                                  ...      ...    ...                  ...   
140505  509ce14e56d662f93a33c221556bee61   829224   SBUX     Clifford Burrows   
140509  ca5ec5821d6a445f90716a3d9695b490  1393612    DFS         Harit Talwar   
140517  5182311313ab849b40a9a5063ec9b2da  1175454    FLT        Todd W. House   
140520  1c4e649f102fc4f3523b36f94390bd72    14272    BMY    Anthony C. Hooper   
140535  8a25b731a4a44ab8020d41f1beda5ba5  1175454    FLT        Todd W. House   

                                                 position  year   salary  \
0       Senior Vice President, President Diagnostics a...  2021   563500   
1                                 Chief Executive Officer  2021  1280000   
2       Senior Vice President, President Life Sciences...  2021   625000   
3          Senior Vice President, Chief Financial Officer  2021   663500   
4        Senior Vice President, President Cross-Lab Group  2021   495000   
...                                                   ...   ...      ...   
140505        president, Starbucks Coffee Americas and US  2011   678942   
140509                            EVP, President—US Cards  2010  1750000   
140517  President, US Direct Business & Chief Operatin...  2010   275000   
140520  SVP Commercial Operations & President US, Japa...  2010   800000   
140535  President, US Direct Business & Chief Operatin...  2009   192877   

         bonus  stockAwards  optionAwards  nonEquityIncentiveCompensation  \
0            0      1541332             0                          738536   
1            0      9165390             0                         3149714   
2            0      1812285             0                          897589   
3            0      2291271             0                         1007000   
4            0      1249771             0                          686716   
...        ...          ...           ...                             ...   
140505       0      2050337        909284                          633822   
140509  875000            0          9292                               0   
140517   40000       787500       1293779                          110000   
140520       0      2649962             0                         1276236   
140535   63906            0        357000                           36094   

        otherCompensation     total  changeInPensionValueAndDeferredEarnings  \
0                   33534   3262887                                      NaN   
1                   77512  15967631                                      NaN   
2                   13684   3802349                                      NaN   
3                  176196   4711733                                      NaN   
4                  499472   3243927                                      NaN   
...                   ...       ...                                      ...   
140505              53391   4325776                                      NaN   
140509                  0   3531442                                  17150.0   
140517              22487   2528766                                      0.0   
140520             267872   6148512                                      NaN   
140535              10679    660556                                      0.0   

          CEO    CFO  Interim  
0       False  False    False  
1        True  False    False  
2       False  False    False  
3       False   True    False  
4       False  False    False  
...       ...    ...      ...  
140505  False  False    False  
140509  Fa

In [13]:
main_df = execpay_df.merge(condensed_finances_df,left_on=['ticker','year'], right_on=['Ticker','Year'])

In [14]:
main_df

id      cik ticker                 name  \
0      73b3a60ba203743c008330c96b7d8b66  1090872      A             Sam Raha   
1      97393f60cd9f321650e472673daaa70c  1090872      A  Michael R. McMullen   
2      83b9cc2bca477fe8ce23e0ab56e70c66  1090872      A        Jacob Thaysen   
3      7c60bb804071675ae15ec930f6dea190  1090872      A       Robert McMahon   
4      259b710a8befe67c61a2c3dec14f344a  1090872      A    Padraig McDonnell   
...                                 ...      ...    ...                  ...   
28435  7b803e9ff0f142d5b1c36b5fdbaced15  1564708    NWS      Gerson Zweifach   
28436  473bf52703cdbffdc9b311d8dee9101a  1564708    NWS      Bedi Ajay Singh   
28437  c36a1d9990a2218f8d2a09b25fb7cd65  1564708    NWS    Robert J. Thomson   
28438  7d64a65ecf544ffd0ce4d4ada1607c73  1564708    NWS      Gerson Zweifach   
28439  abc0667932c203c041cae8baae64f178  1564708    NWS    K. Rupert Murdoch   

                                                position  year   salary  \
0      Senior Vice President, President Diagnostics a...  2021   563500   
1                                Chief Executive Officer  2021  1280000   
2      Senior Vice President, President Life Sciences...  2021   625000   
3         Senior Vice President, Chief Financial Officer  2021   663500   
4       Senior Vice President, President Cross-Lab Group  2021   495000   
...                                                  ...   ...      ...   
28435                                    General Counsel  2014  1500000   
28436                            Chief Financial Officer  2013   655769   
28437                            Chief Executive Officer  2013   992308   
28438                                    General Counsel  2013        0   
28439                                 Executive Chairman  2013        0   

         bonus  stockAwards  optionAwards  ...     total  \
0            0      1541332             0  ...   3262887   
1            0      9165390             0  ...  15967631   
2            0      1812285             0  ...   3802349   
3            0      2291271             0  ...   4711733   
4            0      1249771             0  ...   3243927   
...        ...          ...           ...  ...       ...   
28435        0            0             0  ...   1500000   
28436   713425       496499             0  ...   2087442   
28437  1000000            0             0  ...   2661463   
28438        0            0             0  ...         0   
28439        0            0             0  ...         0   

       changeInPensionValueAndDeferredEarnings    CEO    CFO  Interim  \
0                                          NaN  False  False    False   
1                                          NaN   True  False    False   
2                                          NaN  False  False    False   
3                                          NaN  False   True    False   
4                                          NaN  False  False    False   
...                                        ...    ...    ...      ...   
28435                                      NaN  False  False    False   
28436                                      NaN  False   True    False   
28437                                      NaN   True  False    False   
28438                                      NaN  False  False    False   
28439                                      NaN  False  False    False   

            Revenue  Gross Profit  Operating Income  Income after Tax  \
0      5.952000e+09  3.193250e+09      1.175500e+09      9.837500e+08   
1      5.952000e+09  3.193250e+09      1.175500e+09      9.837500e+08   
2      5.952000e+09  3.193250e+09      1.175500e+09      9.837500e+08   
3      5.952000e+09  3.193250e+09      1.175500e+09      9.837500e+08   
4      5.952000e+09  3.193250e+09      1.175500e+09      9.837500e+08   
...             ...           ...               ...               ...   
28435  8.548750e+09  3.453500e+09      4.040000e+08      1.545000e

In [15]:
main_df.ticker.value_counts()

A       112
MTCH     99
JCI      92
APA      90
EXC      88
       ... 
CARR     16
CME      14
AMP       5
VTRS      5
OGN       5
Name: ticker, Length: 485, dtype: int64

In [16]:
apple_df = main_df[main_df['ticker']=='AAPL']
apple_df
apple_df_2021 = apple_df[apple_df['year']==2020]
apple_df_2021
rev_2021 = apple_df_2021['Revenue']
rev_2021
# Making sure we can perform calculations with this, looking for 'float' or 'int'
type(rev_2021)

apple_df_2012 = apple_df[apple_df['year']==2012]
rev_2012 = max(apple_df_2012['Revenue'])

subtracted_rev = rev_2021-rev_2012

percent_rev_change = (subtracted_rev/rev_2012)*100

print("Revenue percentage growth over 10 years: ", percent_rev_change)

Revenue percentage growth over 10 years:  117    68.57554
118    68.57554
119    68.57554
120    68.57554
121    68.57554
Name: Revenue, dtype: float64


In [17]:
main_df_ticker_list = main_df['ticker'].unique()
main_df_ticker_list
data_2012 = []
# data_2013 = []
# data_2014 = []
# data_2015 = []
# data_2016 = []
# data_2017 = []
# data_2018 = []
# data_2019 = []
# data_2020 = []
data_2021 = []
data_change_dollars = []
data_change_percentage = []


data = {"Ticker": main_df_ticker_list,
       "Revenue 2012": data_2012,
#         "Revenue 2013": data_2013,
#         "Revenue 2014": data_2014,
#         "Revenue 2015": data_2015,
#         "Revenue 2016": data_2016,
#        "Revenue 2017": data_2017,
#         "Revenue 2018": data_2018,
#         "Revenue 2019": data_2019,
#         "Revenue 2020": data_2020,
        "Revenue 2021": data_2021,
       "Revenue Change $": data_change_dollars,
       "Revenue Change %": data_change_percentage}

for ticker in main_df_ticker_list:
    ticker_df = main_df[main_df["ticker"]==ticker]
    try:
        ticker_df_2012 = ticker_df[ticker_df["year"]==2012]
        rev_2012 = max(ticker_df_2012["Revenue"])
        data_2012.append(rev_2012)
    except:
        data_2012.append("No Data Available")
#     try:
#         ticker_df_2013 = ticker_df[ticker_df["year"]==2013]
#         rev_2013 = max(ticker_df_2013["Revenue"])
#         data_2013.append(rev_2013)
#     except:
#         data_2013.append("No Data Available")
#     try:
#         ticker_df_2014 = ticker_df[ticker_df["year"]==2014]
#         rev_2014 = max(ticker_df_2014["Revenue"])
#         data_2014.append(rev_2014)
#     except:
#         data_2014.append("No Data Available")
#     try:
#         ticker_df_2015 = ticker_df[ticker_df["year"]==2015]
#         rev_2015 = max(ticker_df_2015["Revenue"])
#         data_2015.append(rev_2015)
#     except:
#         data_2015.append("No Data Available")
#     try:
#         ticker_df_2016 = ticker_df[ticker_df["year"]==2016]
#         rev_2016 = max(ticker_df_2016["Revenue"])
#         data_2016.append(rev_2016)
#     except:
#         data_2016.append("No Data Available")
#     try:
#         ticker_df_2017 = ticker_df[ticker_df["year"]==2017]
#         rev_2017 = max(ticker_df_2017["Revenue"])
#         data_2017.append(rev_2017)
#     except:
#         data_2017.append("No Data Available")
#     try:
#         ticker_df_2018 = ticker_df[ticker_df["year"]==2018]
#         rev_2018 = max(ticker_df_2018["Revenue"])
#         data_2018.append(rev_2018)
#     except:
#         data_2018.append("No Data Available")    
#     try:
#         ticker_df_2019 = ticker_df[ticker_df["year"]==2019]
#         rev_2019 = max(ticker_df_2019["Revenue"])
#         data_2019.append(rev_2019)
#     except:
#         data_2019.append("No Data Available")
#     try:
#         ticker_df_2020 = ticker_df[ticker_df["year"]==2020]
#         rev_2020 = max(ticker_df_2020["Revenue"])
#         data_2020.append(rev_2020)
#     except:
#         data_2020.append("No Data Available")
    try:
        ticker_df_2021 = ticker_df[ticker_df['year']==2021]
        rev_2021 = max(ticker_df_2021['Revenue'])
        data_2021.append(rev_2021)
    except:
        data_2021.append("No Data Available")
    try:
        rev_change_dollars = rev_2021 - rev_2012
        data_change_dollars.append(rev_change_dollars)
    except:
        print("No calculation performed")
    try:
        rev_change_percentage = (rev_change_dollars / rev_2012)*100
        data_change_percentage.append(rev_change_percentage)
    except:
        print("No calculation performed")
        
        

new_df = pd.DataFrame(data, index = main_df_ticker_list)
new_df
no_data_2012 = new_df["Revenue 2012"] == "No Data Available" 
no_data_2021 = new_df["Revenue 2021"] == "No Data Available"
no_calc_2012 = new_df.loc[no_data_2012, ["Revenue Change $", "Revenue Change %"]] = "No Calculation Performed"
no_calc_2022 = new_df.loc[no_data_2021, ["Revenue Change $", "Revenue Change %"]] = "No Calculation Performed"
print(new_df.head(15))
print(new_df.shape)



      Ticker         Revenue 2012         Revenue 2021  \
A          A    No Data Available         5952000000.0   
AAPL    AAPL       164687000000.0       354175250000.0   
MSFT    MSFT        72930000000.0       172302750000.0   
AMZN    AMZN        61093000000.0  447553750000.000061   
TSLA    TSLA          413256000.0        44618250000.0   
GOOGL  GOOGL    No Data Available    No Data Available   
GOOG    GOOG        46039000000.0       228366750000.0   
UNH      UNH       110618000000.0       275483250000.0   
JNJ      JNJ        67224000000.0        89659750000.0   
XOM      XOM  480681000000.000122    No Data Available   
JPM      JPM       108074000000.0       127989000000.0   
NVDA    NVDA    No Data Available        20525500000.0   
PG        PG        81646000000.0        76618000000.0   
V          V        10720000000.0        23395750000.0   
CVX      CVX       241909000000.0       129147500000.0   

               Revenue Change $          Revenue Change %  
A      No C

The following code block shows that we have 84 of our 485 companies that do not have data in either 2021 or 2012. Some have both that are missing but overall, we can see that 2012 is the bigger culprit of missing revenue data.

In [18]:
print(f"Missing {new_df['Revenue 2012'].value_counts()['No Data Available']} values in 2012")
print(f"Missing {new_df['Revenue 2021'].value_counts()['No Data Available']} values in 2021")
print(f"Missing {new_df['Revenue Change $'].value_counts()['No Calculation Performed']} change history records overall")

Missing 73 values in 2012
Missing 14 values in 2021
Missing 84 change history records overall


In [19]:
index_no_calc = new_df[new_df['Revenue Change $'] == 'No Calculation Performed'].index
new_df.drop(index_no_calc , inplace=True)
new_df
# print(new_df_filtered.head(15))
# print(new_df_filtered.shape)

Ticker    Revenue 2012         Revenue 2021     Revenue Change $  \
AAPL   AAPL  164687000000.0       354175250000.0       189488250000.0   
MSFT   MSFT   72930000000.0       172302750000.0        99372750000.0   
AMZN   AMZN   61093000000.0  447553750000.000061  386460750000.000061   
TSLA   TSLA     413256000.0        44618250000.0        44204994000.0   
GOOG   GOOG   46039000000.0       228366750000.0       182327750000.0   
...     ...             ...                  ...                  ...   
ALK     ALK    4657000000.0         4455250000.0         -201750000.0   
DVA     DVA    8186280000.0        11573402000.0         3387122000.0   
VNO     VNO    2649217100.0         1524119500.0        -1125097600.0   
DISH   DISH   13181334000.0        17680414750.0         4499080750.0   
RL       RL    6924400000.0         5318200000.0        -1606200000.0   

     Revenue Change %  
AAPL       115.059628  
MSFT       136.257713  
AMZN       632.577791  
TSLA     10696.757942  
GOOG        396.02891  
...               ...  
ALK         -4.332188  
DVA         41.375594  
VNO         -42.46906  
DISH        34.132211  
RL         -23.196234  

[401 rows x 5 columns]

In [20]:
# Performing a rank on who grew the most and least, by percentage growth not overall growth
new_df['Growth_Rank'] = new_df['Revenue Change %'].rank(ascending=False)
new_df

# Sorting the rank
rslt_df = new_df.sort_values(by = 'Growth_Rank')
# There are three rows with NaN for 2021 Revenue data to drop
rslt_df = rslt_df.dropna()

In [21]:
rslt_df.head(30)

Ticker   Revenue 2012         Revenue 2021     Revenue Change $  \
TSLA   TSLA    413256000.0        44618250000.0        44204994000.0   
FANG   FANG     74962000.0         4948250000.0         4873288000.0   
DXCM   DXCM     99900000.0         2241049950.0         2141149950.0   
NOW     NOW    243712000.0         5357500000.0         5113788000.0   
CNC     CNC   8110000000.0       120285750000.0       112175750000.0   
INCY   INCY    297059000.0         2830670425.0         2533611425.0   
REGN   REGN   1378477100.0        12799075000.0        11420597900.0   
NFLX   NFLX   3609281800.0        28076899750.0        24467617950.0   
EPAM   EPAM    433799000.0         3239886500.0         2806087500.0   
AMZN   AMZN  61093000000.0  447553750000.000061  386460750000.000061   
CBOE   CBOE    512338000.0         3483075150.0         2970737150.0   
CHTR   CHTR   7504000000.0        50411000000.0        42907000000.0   
ALGN   ALGN    560041000.0         3499667425.0         2939626425.0   
ABMD   ABMD    151794000.0          939890500.0          788096500.0   
LEN     LEN   4105132000.0        25099789500.0        20994657500.0   
CI       CI  29119000000.0       168514000000.0       139395000000.0   
FTNT   FTNT    533639000.0         3026574925.0         2492935925.0   
DHI     DHI   4722500000.0        26726099000.0        22003599000.0   
SIVB   SIVB    984264100.0         5356748000.0         4372483900.0   
ZBRA   ZBRA    996168000.0         5249750000.0         4253582000.0   
CSGP   CSGP    349936000.0         1839697325.0         1489761325.0   
ENPH   ENPH    216678000.0         1137001250.0          920323250.0   
ICE     ICE   1363000000.0         7075250000.0         5712250000.0   
MPWR   MPWR    213813000.0         1071375250.0          857562250.0   
GOOG   GOOG  46039000000.0       228366750000.0       182327750000.0   
TMUS   TMUS  15961590000.0        79039250000.0        63077660000.0   
LRCX   LRCX   3168549000.0        15038472000.0        11869923000.0   
VRTX   VRTX   1527042000.0         6950734500.0         5423692500.0   
MOH     MOH   5914209000.0        24585500000.0        18671291000.0   
MCHP   MCHP   1490390000.0         5904900000.0         4414510000.0   

     Revenue Change %  Growth_Rank  
TSLA     10696.757942          1.0  
FANG      6501.011179          2.0  
DXCM      2143.293243          3.0  
NOW       2098.291426          4.0  
CNC       1383.178175          5.0  
INCY       852.898389          6.0  
REGN       828.493843          7.0  
NFLX       677.908218          8.0  
EPAM       646.863524          9.0  
AMZN       632.577791         10.0  
CBOE       579.839315         11.0  
CHTR        571.78838         12.0  
ALGN        524.89486         13.0  
ABMD       519.188176         14.0  
LEN        511.424663         15.0  
CI          478.70806         16.0  
FTNT       467.157746         17.0  
DHI        465.931159         18.0  
SIVB       444.238889         19.0  
ZBRA       426.994443         20.0  
CSGP        425.72394         21.0  
ENPH        424.74236         22.0  
ICE         419.09391         23.0  
MPWR         401.0805         24.0  
GOOG        396.02891         25.0  
TMUS       395.184064         26.0  
LRCX       374.616993         27.0  
VRTX        355.17638         28.0  
MOH        315.702252         29.0  
MCHP       296.198311         30.0

In [22]:
rslt_df[rslt_df['Revenue Change %']< 0].count()

Ticker              70
Revenue 2012        70
Revenue 2021        70
Revenue Change $    70
Revenue Change %    70
Growth_Rank         70
dtype: int64

In [23]:
print(f"Median of Percent Revenue Change is {rslt_df['Revenue Change %'].median()}")
print(f"Mean of Percent Revenue Change is {rslt_df['Revenue Change %'].mean()}")

Median of Percent Revenue Change is 52.98032445273718
Mean of Percent Revenue Change is 148.36562357985858


In [24]:
rslt_df['Label'] = ['Growing Company' if x >= 0
                    else 'Declining Company' 
                    for x in rslt_df['Revenue Change %']]
rslt_df['Label'].value_counts()

Growing Company      328
Declining Company     70
Name: Label, dtype: int64

In [25]:
main_df.head(15)
main_df = main_df.drop_duplicates()
main_df.loc[main_df['ticker']=='A']

id      cik ticker                   name  \
0    73b3a60ba203743c008330c96b7d8b66  1090872      A               Sam Raha   
1    97393f60cd9f321650e472673daaa70c  1090872      A    Michael R. McMullen   
2    83b9cc2bca477fe8ce23e0ab56e70c66  1090872      A          Jacob Thaysen   
3    7c60bb804071675ae15ec930f6dea190  1090872      A         Robert McMahon   
4    259b710a8befe67c61a2c3dec14f344a  1090872      A      Padraig McDonnell   
10   6a9fd7caafd5ff9409e40f8c289e1cd3  1090872      A          Jacob Thaysen   
11   ccef820f858971ef639e1266f4b96840  1090872      A    Michael R. McMullen   
12   0afb57350eef3b644a4f9802490ce25f  1090872      A         Robert McMahon   
13   5c5621b65ca11ff037e274c80b191cd2  1090872      A               Sam Raha   
14   4deeb4d5b673e6f653a86a1eddda5990  1090872      A         Genomics Group   
15   9637116a014dc78e135a1d65a0831e78  1090872      A              Mark Doak   
22   e22445c4e02f5347b2e40cbee5789855  1090872      A    Michael R. McMullen   
23   ca8648d0544fa3180a6edc2d02a14c82  1090872      A         Robert McMahon   
24   56ef5c9c8b7f5e16ed1c7b2fe79b59d6  1090872      A         Genomics Group   
25   d41442da4b3801c2139f69932b802b1e  1090872      A              Mark Doak   
26   93f8d0fe6316e6a144439bc3c141bc9d  1090872      A          Jacob Thaysen   
27   8830ad897377808bae9e8833e0265300  1090872      A           Michael Tang   
34   55f648eec48f4dcede4ca6a5994f0481  1090872      A          Jacob Thaysen   
35   392901ea2763dd2e201a44dde00f7da9  1090872      A         Robert McMahon   
36   99164580887f667c5d56319a065a07e1  1090872      A    Michael R. McMullen   
37   93f6834cefea5c3dc174d07a9d617864  1090872      A        Human Resources   
38   8da1e032270355f6a9421f9c86c78437  1090872      A              Mark Doak   
39   a8ee964000ed2eb96c1e9cee384182e8  1090872      A         Dominique Grau   
46   e048daf0786cf01a1b260e2d48fc6b99  1090872      A        Human Resources   
47   2a318ae1fb2ac866af17486e49563820  1090872      A    Michael R. McMullen   
48   b846615fbe785357bd5a5c48b468696f  1090872      A     Patrick Kaltenbach   
49   9d5fef9b9e9e99e038903c1255f80895  1090872      A          Didier Hirsch   
50   8f0626122dd54a0ed7e4dde3d5181b14  1090872      A              Mark Doak   
51   16f1ec86b31b1ebced16e98aa3b3b71d  1090872      A          Jacob Thaysen   
58   a13b28e1a078d7d7abcbaa143b74bc66  1090872      A          Didier Hirsch   
59   afeb892081434400721bc17149de2de4  1090872      A          Jacob Thaysen   
60   2d37dac047db8fd03cfcacfe73d701a3  1090872      A  Applied Markets Group   
61   365622904851a1079530120efd7db657  1090872      A        Human Resources   
62   6e525b30af9d2f4e44b7c1d67f072833  1090872      A    Michael R. McMullen   
63   ece95b49afb2267f963e22254bb3749d  1090872      A              Mark Doak   
64   733eaae28284cb81dd47193f5edf955f  1090872      A     Patrick Kaltenbach   
72   bd787dd65f1458a8a34a4d8eb1edf173  1090872      A     Patrick Kaltenbach   
73   88ba9193971115c64b4b338c80ae2369  1090872      A          Didier Hirsch   
74   73bacd1b846e148680eb4a9becc62344  1090872      A                  Chain   
75   1fa34d6f5935e9de3d36e3fa398b8ce3  1090872      A    Michael R. McMullen   
76   8338bde68a45b23cb89207f7f0ca48ad  1090872      A              Mark Doak   
77   e96906309e246a7c5bb160c798988614  1090872      A                  Group   
84   05a042698e63b1f1d6e5e326c5791dfd  1090872      A          Didier Hirsch   
85   605f6d5e14e301ffec5c6ba2041d28b4  1090872      A         Marie Oh Huber   
86   74d8234da776693b7fb0888c50653825  1090872      A    Michael R. McMullen   
87   d3af951ba9570439f8e44affe16bc27f  1090872      A    Ronald S. Nersesian   
88   56fe7f35486e52fd83e1dded8482b048  1090872      A                  Group   
89   a6ad6be0d2312415b499e2961d5f1589  1090872      A    William P. Sullivan   
96   649b6806fbe4a654140ba6e0aa7c2c25  1090872      A                  Group   
97   b4add3c41ed5c86ed22bfce5309856

In [26]:
# Need to drop duplicate Agilent ones with the name "Group" or "human resources"
indexAge = main_df[ (main_df['name'] == 'Group') | 
                   (main_df['name'] == 'Human Resources') ].index
main_df.drop(indexAge , inplace=True)
df_CEO = main_df[(main_df['CEO'] == True)] 
df_CFO = main_df[(main_df['CFO'] == True)] 

df_CEO = df_CEO.groupby('ticker').sum()
df_CFO = df_CFO.groupby('ticker').sum()

# Subsetting for the fields we want
df_CEO = df_CEO[['CEO','salary', 'bonus', 'stockAwards', 
                 'optionAwards','nonEquityIncentiveCompensation', 
                 'total', 'Revenue', 'Net Income Common']]
df_CEO.loc[df_CEO["CEO"] > 0, "CEO"] = True

df_CFO = df_CFO[['CEO','salary', 'bonus', 'stockAwards', 
                 'optionAwards','nonEquityIncentiveCompensation', 
                 'total', 'Revenue', 'Net Income Common']]
df_CFO.loc[df_CFO["CEO"] == 0, "CEO"] = False
df_CFO

C:\Users\reagins\AppData\Local\Temp\ipykernel_20348\555004581.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df.drop(indexAge , inplace=True)


CEO    salary    bonus  stockAwards  optionAwards  \
ticker                                                        
A       False   5085129  3000000     19646031       2999887   
AAL     False   7218184        0     38471191             0   
AAP     False   6083854   980000     11459057       2125173   
AAPL    False  10355499        0    224247705             0   
ABBV    False   8307776        0     28358014       7884662   
...       ...       ...      ...          ...           ...   
YUM     False   7267423  1214000     12410093      12570321   
ZBH         5   8256718  2240321     22413931      20588689   
ZBRA    False   7106988        0     13943450       3185978   
ZION    False   6419692  3652500      5839982        985436   
ZTS     False   7704090  2600000     18872778       7109989   

        nonEquityIncentiveCompensation      total       Revenue  \
ticker                                                            
A                              4580051   36893668  4.623975e+10   
AAL                           11200278   62707472  4.588178e+11   
AAP                            3206254   25343797  1.189908e+11   
AAPL                          31428755  266979198  2.529333e+12   
ABBV                          15018849   78579417  2.689395e+11   
...                                ...        ...           ...   
YUM                            9141876   49258979  9.790075e+10   
ZBH                            7456420   64576959  9.811212e+10   
ZBRA                           7189476   31782137  5.131666e+10   
ZION                           1808345   17642731  2.865551e+10   
ZTS                            6657597   45597108  8.033900e+10   

        Net Income Common  
ticker                     
A            6.070500e+09  
AAL          1.355525e+10  
AAP          6.034603e+09  
AAPL         5.686565e+11  
ABBV         4.946950e+10  
...                   ...  
YUM          1.473400e+10  
ZBH          8.603825e+09  
ZBRA         3.526936e+09  
ZION         5.211822e+09  
ZTS          1.365375e+10  

[484 rows x 9 columns]

In [34]:
frames = [df_CEO, df_CFO]
  
new_main = pd.concat(frames)
new_main = new_main.sort_values(by=['ticker'])

new_main['salary/revenue'] = new_main['salary'] / new_main['Revenue']
new_main['bonus/revenue'] = new_main['bonus'] / new_main['Revenue']
new_main['stockAwards/revenue'] = new_main['stockAwards'] / new_main['Revenue']
new_main['optionAwards/revenue'] = new_main['optionAwards'] / new_main['Revenue']
new_main['nonEquityIncentiveCompensation/revenue'] = new_main['nonEquityIncentiveCompensation'] / new_main['Revenue']
new_main['all_Equity_Awards'] = new_main['stockAwards'] + new_main['optionAwards']

## Dropping Where CEO is not equal to True or False
indexNumbers = new_main[ (new_main['CEO'] == 3) | 
                 (new_main['CEO'] == 2) | (new_main['CEO'] == 5)].index
new_main.drop(indexNumbers , inplace=True)
#
## Converting True/False CEO column from object to boolean because the model
## was not recognizing the y-predictor type.
new_main['CEO'] = new_main['CEO'].astype(bool)
print(new_main.dtypes)
new_main

CEO                                          bool
salary                                      int64
bonus                                       int64
stockAwards                                 int64
optionAwards                                int64
nonEquityIncentiveCompensation              int64
total                                       int64
Revenue                                   float64
Net Income Common                         float64
salary/revenue                            float64
bonus/revenue                             float64
stockAwards/revenue                       float64
optionAwards/revenue                      float64
nonEquityIncentiveCompensation/revenue    float64
all_Equity_Awards                           int64
dtype: object


CEO    salary     bonus  stockAwards  optionAwards  \
ticker                                                         
A        True  11458269         0     68552878      15107596   
A       False   5085129   3000000     19646031       2999887   
AAL      True   2441140         0    130227654             0   
AAL     False   7218184         0     38471191             0   
AAP      True   9668318   3485000     44487383      14649081   
...       ...       ...       ...          ...           ...   
ZBRA     True  12679100         0     59775901      12967242   
ZION    False   6419692   3652500      5839982        985436   
ZION     True  21152988  11702000     15629354       3350878   
ZTS      True  10643533         0     51539576      20646680   
ZTS     False   7704090   2600000     18872778       7109989   

        nonEquityIncentiveCompensation      total       Revenue  \
ticker                                                            
A                             16133895  115188782  5.834650e+10   
A                              4580051   36893668  4.623975e+10   
AAL                           10203331  159413533  4.842888e+11   
AAL                           11200278   62707472  4.588178e+11   
AAP                            8633582   83061159  9.050150e+10   
...                                ...        ...           ...   
ZBRA                          19148913  105531711  4.446959e+10   
ZION                           1808345   17642731  2.865551e+10   
ZION                           7071579   58692242  7.047798e+10   
ZTS                           16059793  101710673  5.388575e+10   
ZTS                            6657597   45597108  8.033900e+10   

        Net Income Common  salary/revenue  bonus/revenue  stockAwards/revenue  \
ticker                                                                          
A            7.675250e+09        0.000196       0.000000             0.001175   
A            6.070500e+09        0.000110       0.000065             0.000425   
AAL          1.304125e+10        0.000005       0.000000             0.000269   
AAL          1.355525e+10        0.000016       0.000000             0.000084   
AAP          4.663842e+09        0.000107       0.000039             0.000492   
...                   ...             ...            ...                  ...   
ZBRA         3.304858e+09        0.000285       0.000000             0.001344   
ZION         5.211822e+09        0.000224       0.000127             0.000204   
ZION         1.245729e+10        0.000300       0.000166             0.000222   
ZTS          9.552250e+09        0.000198       0.000000             0.000956   
ZTS          1.365375e+10        0.000096       0.000032             0.000235   

        optionAwards/revenue  nonEquityIncentiveCompensation/revenue  \
ticker                                                                 
A                   0.000259                                0.000277   
A                   0.000065                                0.000099   
AAL                 0.000000                                0.000021   
AAL                 0.000000                                0.000024   
AAP                 0.000162                                0.000095   
...                      ...                                     ...   
ZBRA                0.000292                                0.000431   
ZION                0.000034                                0.000063   
ZION                0.000048                                0.000100   
ZTS                 0.000383                                0.000298   
ZTS                 0.000088                                0.000083   

        all_Equity_Awards  
ticker                     
A                83660474  
A                22645918  
AAL             130227654  
AAL              38471191  
AAP              59136464  
...                   ...  
ZBRA             72743143  
ZION              6825418  
ZION             18980232  
ZTS   

In [28]:
new_main.to_csv('new_main_df.csv',index=False)

Given that we have a small data set here, I am going to go off the guidance of the book in partitioning. "On the other hand, for smaller or less complex data sets, one should retain sufficient records for accurate assessment, so that the training sets would contain only 50–67% of the original data." I think we should sufficient records to train on though, so I posit that we do a 60% training, 40% testing split (if you think we should do otherwise, by all means I am all ears).

In [29]:
#Partitioning the data
fin_train, fin_test = train_test_split(new_main, 
                                       test_size = 0.4, random_state = 7)

# Testing to see if we have partitioned correctly.
print("Shape of entire dataset: ",new_main.shape,"\n")
print("Value counts of CEO vs CFO:\n",new_main['CEO'].value_counts(),"\n")
print("Shape of training dataset: ",fin_train.shape,"\n")
print("Shape of testing dataset: ",fin_test.shape)

Shape of entire dataset:  (959, 15) 

Value counts of CEO vs CFO:
 True     497
False    462
Name: CEO, dtype: int64 

Shape of training dataset:  (575, 15) 

Shape of testing dataset:  (384, 15)


Baseline model: We call the "positive" class "True" for CEO and see that a baseline model would predict correctly with (497/959) = 52% accuracy. If our model beats 52% accuracy, then it beats the baseline and is considered useful. I am not going to choose the X variables as proportions of revenue, although I can see there being multicollinearity concerns with this approach if we are incorporating the Y variable into the X variable in this way. I am curious to hear your thoughts on this. Note that I am not taking the "total comp" as this would definitely be a multicollinearity concern if we are using the other 5 variables.

In [30]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report

In [31]:
# CART Model
y_train = fin_train[['CEO']]
X_train = fin_train[['salary/revenue','bonus/revenue','stockAwards/revenue','optionAwards/revenue',
              'nonEquityIncentiveCompensation/revenue']]

# Setting up the variables from the test dataset.
y_test = fin_test['CEO']
X_test = fin_test[['salary/revenue','bonus/revenue','stockAwards/revenue','optionAwards/revenue',
              'nonEquityIncentiveCompensation/revenue']]

#######################
#scale the data
sc = MinMaxScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)
#######################

#instantiate and fit DT classifier
cart01 = DecisionTreeClassifier(criterion = "gini", max_leaf_nodes=5).fit(X_train_sc,y_train)

#make predictions with training dataset
y_predCart_train = cart01.predict(X_train_sc)

print("Confusion matrix for training data:")
print(confusion_matrix(y_true=y_train, y_pred=y_predCart_train))
print(classification_report(y_true=y_train, y_pred=y_predCart_train))

#make predictions with test dataset
y_predCart_test = cart01.predict(X_test_sc)

print("\nConfusion matrix for testing data:")
print(confusion_matrix(y_true=y_test_cart, y_pred=y_predCart_test))
print(classification_report(y_true=y_test_cart, y_pred=y_predCart_test))

#data_crosstab_cart = pd.crosstab(y_test,
#                            predCart_test, 
#                            rownames=['Actual'], 
#                            colnames=['Prediction'],
#                            margins = False)
#print(data_crosstab_cart)

Confusion matrix for training data:
[[174  97]
 [ 55 249]]
              precision    recall  f1-score   support

       False       0.76      0.64      0.70       271
        True       0.72      0.82      0.77       304

    accuracy                           0.74       575
   macro avg       0.74      0.73      0.73       575
weighted avg       0.74      0.74      0.73       575


Confusion matrix for testing data:


NameError: name 'y_test_cart' is not defined

In [ ]:
#C5.0 Model
c50_01 = DecisionTreeClassifier(criterion="entropy", max_leaf_nodes=5).fit(X_train_sc,y_train)

#make predictions with training dataset
y_predC5_train = c50_01.predict(X_train_sc)

print("Confusion matrix for training data:")
print(confusion_matrix(y_true=y_train, y_pred=y_predC5_train))
print(classification_report(y_true=y_train, y_pred=y_predC5_train))

#make predictions with training dataset
y_predC5_test = c50_01.predict(X_test_sc)

print("Confusion matrix for training data:")
print(confusion_matrix(y_true=y_test, y_pred=y_predC5_test))
print(classification_report(y_true=y_test, y_pred=y_predC5_test))




#predC5_test = c50_01.predict(X_test)
#data_crosstab_C5 = pd.crosstab(y_test,
#                            predC5_test, 
#                            rownames=['Actual'], 
#                            colnames=['Prediction'],
#                            margins = False)
#print(data_crosstab_C5)

In [ ]:
#Random Forest Model
rfy = np.ravel(y)
rf01 = RandomForestClassifier(n_estimators = 100, criterion="gini").fit(X,rfy)
predRf = rf01.predict(X)

predRf_test = rf01.predict(X_test)
data_crosstab_RF = pd.crosstab(y_test,
                            predRf_test, 
                            rownames=['Actual'], 
                            colnames=['Prediction'],
                            margins = False)
print(data_crosstab_RF)